In [103]:
from DbConnector import DbConnector
from part2 import Database 
from tabulate import tabulate

# Set up the program
from dotenv import load_dotenv
import os
import numpy as np
from haversine import haversine, Unit

load_dotenv()

user = os.getenv('DB_USER')
password = os.getenv('DB_PASSWORD')

db = DbConnector(USER=user, PASSWORD=password)

Using user:  root
Connected to: 8.0.29
You are connected to the database: ('assignment2',)
-----------------------------------------------



### Task 1

How many users, activities and trackpoints are there in the dataset (after it is inserted into the database)

In [104]:
result: list = []

for table_name in ["User", "Activity", "TrackPoint"]:
    query = "SELECT COUNT(*) FROM %s"
    db.cursor.execute(query % table_name)
    result.append(db.cursor.fetchone())

print("Number of Users, Activities and TrackPoints after the dataset has been inserted into the database")
print(tabulate([["Users", "Activities", "TrackPoints"], [result[0][0], result[1][0], result[2][0]]], headers="firstrow"))

Number of Users, Activities and TrackPoints after the dataset has been inserted into the database
  Users    Activities    TrackPoints
-------  ------------  -------------
    182          7877        5355109


### Task 2
Find the average, maximum and minimum number of trackpoints per user

##### Average

In [105]:
# Get the average number of trackpoints per user
query = """SELECT User.id, COALESCE(average, 0) AS average 
                FROM User LEFT JOIN (SELECT user_id, AVG(trackpoints) average 
                FROM (SELECT Activity.user_id , COUNT(t.id) AS trackpoints 
                FROM TrackPoint t JOIN Activity ON t.activity_id = Activity.id 
                JOIN User ON Activity.user_id = User.id GROUP BY Activity.id) 
                AS Trackpoints GROUP BY user_id) a ON a.user_id = User.id"""
db.cursor.execute(query)
rows = db.cursor.fetchall()
print("Average trackpoints per user")
print(tabulate(rows, headers=db.cursor.column_names))


Average trackpoints per user
  id    average
----  ---------
 000   670.871
 001   824.965
 002   924.801
 003   807.387
 004   761.858
 005   587.548
 006   801.125
 007   758.275
 008  1376
 009  1218.32
 010  1107.5
 011   451.756
 012   629.783
 013  1388.3
 014   905.936
 015   806.583
 016   690.222
 017   868.245
 018   540.795
 019   372.291
 020   704.638
 021   475
 022   900.695
 023   979.312
 024   860.408
 025   606.295
 026  1034.71
 027  2037
 028  1195.36
 029  1713.17
 030   871.352
 031   489.5
 032  1190.43
 033   735.25
 034   558.678
 035  1449.71
 036   803.75
 037   890.977
 038  1071.51
 039   756.51
 040   953.55
 041   797.416
 042   555.946
 043   795.094
 044   628.492
 045   752.125
 046   703.807
 047    65.8333
 048  1118
 049     0
 050   760.833
 051   231.3
 052   177
 053     0
 054  1093
 055   132.526
 056    39.7333
 057   127
 058    38.5
 059     0
 060     8
 061   140
 062   439.054
 063   709.857
 064   695
 065  1114.94
 066   889.375
 067  

##### Maximum

In [106]:
# Get the maximum number of trackpoints per user
query = """SELECT User.id, COALESCE(maximum, 0) AS maximum 
                FROM User LEFT JOIN (SELECT user_id, MAX(trackpoints) AS maximum 
                FROM (SELECT Activity.user_id , COUNT(t.id) AS trackpoints FROM TrackPoint t 
                INNER JOIN Activity ON t.activity_id = Activity.id GROUP BY Activity.id) 
                AS Trackpoints GROUP BY user_id) a ON a.user_id = User.id"""
db.cursor.execute(query)
rows = db.cursor.fetchall()
print("Maximum trackpoints per user")
print(tabulate(rows, headers=db.cursor.column_names))

Maximum trackpoints per user
  id    maximum
----  ---------
 000       2359
 001       2472
 002       2438
 003       2485
 004       2482
 005       2058
 006       2478
 007       2228
 008       2499
 009       2396
 010       1964
 011       2306
 012       2277
 013       2486
 014       2499
 015       2411
 016       2360
 017       2471
 018       2245
 019       2276
 020       2201
 021        475
 022       2421
 023       2215
 024       2377
 025       2464
 026       2396
 027       2480
 028       2477
 029       2440
 030       2454
 031       1254
 032       2358
 033       1205
 034       2448
 035       2493
 036       2299
 037       2421
 038       2500
 039       2464
 040       2434
 041       2491
 042       2350
 043       2005
 044       2206
 045       1932
 046       2457
 047        227
 048       1118
 049          0
 050       2484
 051        951
 052        177
 053          0
 054       1527
 055        670
 056        165
 057        510
 058       

##### Minimum

In [107]:
# Get the minimum number of trackpoints per user
query = """
        SELECT User.id, COALESCE(minimum, 0) AS minimum FROM User LEFT JOIN (SELECT user_id, MIN(trackpoints) AS minimum
            FROM (SELECT Activity.user_id , COUNT(t.id) AS trackpoints 
            FROM TrackPoint t
            INNER JOIN Activity ON t.activity_id = Activity.id 
            GROUP BY Activity.id) 
            AS Trackpoints 
            GROUP BY user_id) a ON a.user_id = User.id
            """
db.cursor.execute(query)
rows = db.cursor.fetchall()
print("Minimum trackpoints per user")
print(tabulate(rows, headers=db.cursor.column_names))

Minimum trackpoints per user
  id    minimum
----  ---------
 000          5
 001         33
 002          4
 003          3
 004          4
 005          5
 006         14
 007          6
 008        165
 009        134
 010        663
 011          3
 012         64
 013         13
 014          3
 015         22
 016          7
 017          4
 018          3
 019         11
 020         22
 021        475
 022         15
 023         31
 024          4
 025          3
 026          9
 027       1760
 028         32
 029          4
 030          9
 031         23
 032         62
 033         39
 034          4
 035        200
 036         13
 037         14
 038          8
 039          7
 040         39
 041          3
 042          4
 043         26
 044          3
 045         71
 046        175
 047          4
 048       1118
 049          0
 050          3
 051          5
 052        177
 053          0
 054        659
 055          5
 056         16
 057          9
 058       

### Task 3

Find the top 15 users with the highest number of activities.

In [108]:
query = """
    SELECT RANK() OVER (
        ORDER BY COUNT(*) DESC
    ) Top, user_id, COUNT(*) as num_of_activities FROM Activity GROUP BY user_id ORDER BY COUNT(*) DESC LIMIT 15
    """
db.cursor.execute(query)
rows = db.cursor.fetchall()
columns = db.cursor.column_names
print("Top 15 users with the highest number of activities")
print(tabulate(rows, headers=columns))

Top 15 users with the highest number of activities
  Top    user_id    num_of_activities
-----  ---------  -------------------
    1        025                  715
    2        128                  519
    3        062                  406
    4        041                  399
    5        004                  346
    6        140                  345
    7        017                  265
    8        003                  261
    9        014                  236
   10        030                  210
   11        011                  201
   12        039                  198
   13        034                  180
   14        000                  155
   15        002                  146


### Task 4

Find all users who have taken a bus.

In [109]:
# Get the users who have taken the bus
query = """
SELECT DISTINCT user_id
FROM Activity
WHERE transportation_mode = 'bus'
"""
db.cursor.execute(query)
users = db.cursor.fetchall()

table = [["User id"]]
table.extend([user for user in users])
print("All users who have taken a bus")
print(tabulate(table, headers="firstrow"))

All users who have taken a bus
  User id
---------
      010
      052
      062
      073
      081
      084
      085
      091
      092
      112
      125
      128
      175


### Task 5

List the top 10 users by their amount of different transportation modes.

In [110]:
query = """SELECT RANK() OVER (
        ORDER BY COUNT(DISTINCT(transportation_mode)) DESC
        ) AS Top, user_id, COUNT(DISTINCT(transportation_mode)) as DifferentTransportation 
                    FROM Activity GROUP BY user_id ORDER BY DifferentTransportation DESC LIMIT 10;"""
db.cursor.execute(query)
rows = db.cursor.fetchall()
print("Top 10 users by their amount of different transportation modes")
print(tabulate(rows, headers=columns))

Top 10 users by their amount of different transportation modes
  Top    user_id    num_of_activities
-----  ---------  -------------------
    1        128                    9
    2        062                    7
    3        085                    4
    4        058                    3
    4        078                    3
    4        081                    3
    4        084                    3
    4        112                    3
    4        163                    3
   10        010                    2


### Task 6

Find activities that are registered multiple times. You should find the query even
if it gives zero result.

Assumption: Registered multiple times, meaning duplicates with the same user_id, transportation_mode, start_date_time, end_date_time.

In [111]:
query = """
    SELECT a.id FROM Activity AS a WHERE EXISTS 
        (SELECT b.id FROM Activity AS b WHERE a.user_id = b.user_id 
        AND a.transportation_mode = b.transportation_mode AND 
        a.start_date_time = b.start_date_time AND a.end_date_time = b.end_date_time 
        AND a.id != b.id)
    """
db.cursor.execute(query)
rows = db.cursor.fetchall()
columns = db.cursor.column_names
print("Activities that are registered multiple times")
print(tabulate(rows, headers=columns))

Activities that are registered multiple times
id
----


### Task 7

a) Find the number of users that have started an activity in one day and ended the activity the next day.

In [112]:
query = """
SELECT COUNT(DISTINCT user_id) 
FROM Activity 
WHERE DATEDIFF(end_date_time, start_date_time) > 0;
"""
db.cursor.execute(query)
num_users = db.cursor.fetchone()

print("Number of users with activity that ends the next day")
print(tabulate([["Number of users"], [num_users[0]]], headers="firstrow"))

Number of users with activity that ends the next day
  Number of users
-----------------
               66


b) List the transportation mode, user id and duration for these activities.

Assumption: As the question explicitly mentions transportation mode we do not count the rows in the Activity table with transportation mode equals NULL as relevant to this question. Thus, there are less rows in this sub task.

In [113]:
query = """
SELECT 
    user_id, 
    transportation_mode, 
    SEC_TO_TIME(TIMESTAMPDIFF(SECOND, start_date_time, end_date_time))
FROM Activity 
WHERE DATEDIFF(end_date_time, start_date_time) > 0
AND transportation_mode IS NOT NULL;
"""
db.cursor.execute(query)
users_info = db.cursor.fetchall()

table = [["User", "Transportation Mode", "Duration"]]
content = [[user[0], user[1], user[2]] for user in users_info]
table.extend(content)

print(tabulate(table, headers="firstrow"))

  User  Transportation Mode    Duration
------  ---------------------  ----------
   020  bike                   10:10:01
   021  walk                   3:57:13
   058  car                    0:36:12
   062  walk                   1:23:44
   085  bus                    0:58:25
   115  car                    1:28:04
   115  car                    1:02:50
   115  car                    1:00:56
   115  car                    1:05:10
   115  car                    1:23:14
   115  car                    0:54:09
   115  car                    1:08:43
   115  car                    1:09:36
   115  car                    1:03:47
   115  car                    1:01:42
   115  car                    0:55:50
   115  car                    1:24:08
   115  car                    1:17:13
   115  car                    1:01:33
   115  car                    1:03:29
   115  car                    0:58:19
   115  car                    7:46:29
   115  car                    1:05:39
   115  car         

### Task 8

Find the number of users which have been close to each other in time and space. Close is defined as the same space (50 meters) and for the same half minute (30 seconds)

In [114]:
min_max_lat_lon_query = """SELECT MIN(lat), MAX(lat), MIN(lon), MAX(lat)
                FROM trackpoint"""

db.cursor.execute(min_max_lat_lon_query)
min_max = db.cursor.fetchall()

min_lat = min_max[0][0] - 0.1
max_lat = min_max[0][1] + 0.1
min_lon = min_max[0][2] - 0.1
max_lon = min_max[0][3] + 0.1


lat = np.linspace(min_lat,max_lat, 5)
lon = np.linspace(min_lon,max_lon, 5)



            # Divinding trackpoints into smaller areas
query = """WITH limitedTrackPoints AS 
                (SELECT * 
                FROM trackpoint 
                WHERE lat > %s 
                AND lat < %s 
                AND lon > %s 
                AND lon < %s)
                
                SELECT DISTINCT *
                FROM Activity a WHERE EXISTS ( 
                SELECT * FROM limitedTrackPoints t1 WHERE EXISTS(
                    SELECT * FROM limitedTrackPoints t2 WHERE
                        t2.lon < t1.lon + 0.0040
                        AND t2.lat < t1.lat + 0.0040
                        AND t1.date_time BETWEEN t2.date_time - INTERVAL '30' SECOND AND t2.date_time + INTERVAL '30' SECOND
                        )
                )GROUP BY a.user_id
                """


users = []
for i in range(len(lat)-1):
    for j in range(len(lon)-1):
        #print(str((lat[i], lat[i + 1], lon[j], lon[j + 1])))
        db.cursor.execute(query, (lat[i], lat[i + 1], lon[j], lon[j + 1]))
        rows = db.cursor.fetchall()

            # Checks if there is content in output
        if len(rows) > 0:
            # Adds userid to list
            for row in rows:
                users.append(row[1])
    # Removes duplicates
users = np.unique(users)
print(tabulate(users, db.cursor.column_names))

  id    user_id    transportation_mode
----  ---------  ---------------------
   0          0                      0
   0          0                      1
   0          0                      2
   0          0                      3
   0          0                      4
   0          0                      5
   0          0                      6
   0          0                      7
   0          0                      8
   0          0                      9
   0          1                      0
   0          1                      1
   0          1                      2
   0          1                      3
   0          1                      4
   0          1                      5
   0          1                      6
   0          1                      7
   0          1                      8
   0          1                      9
   0          2                      0
   0          2                      1
   0          2                      2
   0          2          

### Task 9

Find the top 15 users who have gained the most altitude meters.

In [115]:
# 1. Retrieve all the trackpoints which is valid
# 2. Calculate the difference between the current trackpoint's altitude and the previous one
# 3. Calculate the sum of difference if the difference is postive based on the tip given in task for each activity
# 4. Sum each user's altitude positive differences in feet with 0.304 to convert it to meters

query = """SELECT RANK() OVER (
        ORDER BY SUM(activity_altitude)*0.304 DESC
        ) AS Top, user_id, SUM(activity_altitude)*0.304 as altitude_in_meters FROM Activity JOIN 
        (SELECT activity_id, SUM(difference) AS activity_altitude FROM 
        (SELECT activity_id, altitude - LAG(altitude) OVER 
        (PARTITION BY activity_id ORDER BY date_time) AS difference FROM 
        (SELECT activity_id, altitude, date_time FROM TrackPoint WHERE altitude != -777) AS t) 
        AS altitude_difference WHERE difference > 0 GROUP BY activity_id) AS difference_table 
        ON Activity.id = difference_table.activity_id GROUP BY user_id ORDER BY altitude_in_meters DESC 
        LIMIT 15"""
db.cursor.execute(query)
rows = db.cursor.fetchall()
print("Top 15 users who have gained the most altitude meters")
print(tabulate(rows, headers=db.cursor.column_names))

Top 15 users who have gained the most altitude meters
  Top    user_id    altitude_in_meters
-----  ---------  --------------------
    1        004              331165
    2        041              240127
    3        003              233050
    4        030              175219
    5        128              152571
    6        039              146319
    7        000              121186
    8        002              114761
    9        025              108862
   10        037               98960.5
   11        140               94589.9
   12        062               74559
   13        017               62402.1
   14        034               61278.5
   15        042               61159


### Task 10

Find the users that have traveled the longest total distance in one day for each
transportation mode.

In [116]:
from haversine import haversine, Unit

query = """
SELECT a.id, tp.lat, tp.lon, a.transportation_mode, a.user_id
FROM Activity a
INNER JOIN TrackPoint tp ON a.id = tp.activity_id 
WHERE (TIMESTAMPDIFF(SECOND, a.start_date_time, a.end_date_time)) < 60 * 60 * 24
AND a.transportation_mode IS NOT NULL;
"""

db.cursor.execute(query)
result = db.cursor.fetchall()

activity = {} # Keep track of the distance of each activity
last_activity_position = {} # Keep track of the last position of each activity
transportation = {} # Keep track of the transportation mode of each activity
activity_user = {} # Keep track of the user of each activity

# Go through all the activities and calculate the distance between the current and last position
for res in result:
    activity_id = res[0]
    current_position = (res[1], res[2])
    last_position = last_activity_position.get(activity_id, current_position)
    last_activity_position[activity_id] = current_position # Update the last position
    distance = haversine(current_position, last_position, unit=Unit.KILOMETERS)
    activity[activity_id] = activity.get(activity_id, 0) + distance # Update the distance
    transportation[activity_id] = res[3]
    activity_user[activity_id] = res[4]

# Find the longest activity for each transportation mode
longest_activity = {}
for act in activity.items():
    transportation_mode = transportation[act[0]]
    longest_activity[transportation_mode] = (
        act[0],
        max(act[1], longest_activity.get(transportation_mode, (0, 0))[1]),
    ) # Update the longest activity

# Create a table with the longest activity for each transportation mode
users = []
for longest in longest_activity.items():
    activity_id = longest[1][0]
    transportation_mode = longest[0]
    user_id = activity_user[activity_id]
    distance = activity[activity_id]
    users.append((user_id, transportation_mode, distance))

table = [["User", "Transportation mode", "Distance (km)"]]
table.extend(users)

print(tabulate(table, headers="firstrow"))

  User  Transportation mode      Distance (km)
------  ---------------------  ---------------
   175  bus                          1.80771
   163  taxi                        11.3794
   167  walk                         0.455136
   167  bike                         4.0598
   128  car                         18.5745
   062  run                          0.0332532
   128  train                       19.0923
   128  subway                      10.8537
   128  airplane                  4391.42
   128  boat                        65.5548


### Task 11

Find all users who have invalid activities, and the number of invalid activities per user.

In [117]:
query = """SELECT a.user_id, count(*) FROM Activity a JOIN TrackPoint t1 on a.id = t1.activity_id 
        JOIN TrackPoint t2 on t1.id = t2.id-1 AND t1.activity_id = t2.activity_id 
        WHERE t2.date_time > t1.date_time + INTERVAL 5 MINUTE GROUP BY a.user_id"""
db.cursor.execute(query)
rows = db.cursor.fetchall()
print(tabulate(rows, headers=db.cursor.column_names))

  user_id    count(*)
---------  ----------
      000         442
      001         116
      002         295
      003         839
      004        1169
      005         126
      006          48
      007         133
      008          33
      009         114
      011          38
      012         163
      013          68
      014         328
      015         136
      016          60
      017         492
      018          97
      019          65
      020          15
      021           1
      022         188
      023          30
      024          73
      025         424
      026          51
      027          13
      028          90
      029          37
      030         509
      031           7
      032          36
      033           3
      034         240
      035         122
      036         125
      037         279
      038         264
      039         423
      040          50
      041         386
      042         117
      043          72
      044 

### Task 12

Find all the users who have registered transportation_mode and their most used transportation_mode

Comment on implementation: Although with this dataset, does not need to join User, however in cases where user does not have labels, but still has some activities in Activity with transportation_mode then a join is necessary. For users that have the same number of activities tagged on multiple transportation mode, we've decided to take the first transportation mode in alphabetical order.

In [118]:
# 1. Select and order the transportation mode for each user based on 
query = """SELECT User.id, transportation_mode FROM User LEFT JOIN (SELECT user_id, transportation_mode FROM 
        (SELECT user_id, transportation_mode, ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY COUNT(*) DESC, transportation_mode ASC) 
        AS rownum FROM Activity JOIN User ON Activity.user_id = User.id WHERE transportation_mode IS NOT NULL 
        AND User.has_labels=TRUE GROUP BY user_id, transportation_mode)
        AS activity_grouped WHERE rownum=1) AS new_table ON User.id = user_id WHERE has_labels=TRUE ORDER BY User.id"""
db.cursor.execute(query)
rows = db.cursor.fetchall()
print("Find all the users who have registered transportation_mode and their most used transportation_mode")
print(tabulate(rows, headers=db.cursor.column_names))

Find all the users who have registered transportation_mode and their most used transportation_mode
  id  transportation_mode
----  ---------------------
 010  taxi
 020  bike
 021  walk
 052  bus
 053
 056  bike
 058  car
 059
 060  walk
 062  bus
 064  bike
 065  bike
 067  walk
 068
 069  bike
 073  walk
 075  walk
 076  car
 078  walk
 080  bike
 081  bike
 082  walk
 084  walk
 085  walk
 086  car
 087  walk
 088
 089  car
 091  bus
 092  bus
 096
 097  bike
 098  taxi
 100
 101  car
 102  bike
 104
 105
 106
 107  walk
 108  walk
 110
 111  taxi
 112  walk
 114
 115  car
 116
 117  walk
 118
 124
 125  bike
 126  bike
 128  car
 129
 136  walk
 138  bike
 139  bike
 141
 144  walk
 147
 153  walk
 154
 161  walk
 163  bike
 167  bike
 170
 174
 175  bus
 179


### End

In [119]:
# Closing the connection
db.close_connection()


-----------------------------------------------
Connection to 8.0.29 is closed
